# 06: Model - MLP Neural Network

## Objective
To implement and evaluate a Multi-layer Perceptron (MLP) Neural Network for complex pattern recognition in churn prediction.

## Methodology
1. **Clean Data**: Using `src.preprocessing`.
2. **Training**: MLP with two hidden layers (100, 50) and scaled features.

In [ ]:
# 1. Setup
!pip install -q pandas numpy scikit-learn matplotlib seaborn

import pandas as pd
from src.preprocessing import clean_data

df_raw = pd.read_csv("ecommerce_churn_llm_final.csv")
df = clean_data(df_raw)

from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline

X = df.drop(columns=["customer_id", "churn", "customer_feedback", "support_chat_excerpt", "reason_for_low_activity"], errors="ignore")
y = df["churn"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

categorical_cols = X.select_dtypes(include=['object', 'string', 'category']).columns.tolist()
numerical_cols = X.select_dtypes(include=['number']).columns.tolist()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ]
)

mlp_model = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=500, random_state=42))
])

mlp_model.fit(X_train, y_train)
y_pred = mlp_model.predict(X_test)

print(classification_report(y_test, y_pred))
print(f"ROC AUC: {roc_auc_score(y_test, mlp_model.predict_proba(X_test)[:, 1]):.3f}")